In [1]:
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

import numpy as np
# import cupy as xp

import pyopencl as cl
import time
import os

In [2]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [5]:
gpu_matmul = """
#define TSM 128                // The tile-size in dimension M
#define TSN 128                // The tile-size in dimension N
#define TSK 16                 // The tile-size in dimension K
#define WPTM 8                 // The work-per-thread in dimension M
#define WPTN 8                 // The work-per-thread in dimension N
#define RTSM (TSM/WPTM)        // The reduced tile-size in dimension M
#define RTSN (TSN/WPTN)        // The reduced tile-size in dimension N
#define LPTA ((TSK*TSM)/(RTSM*RTSN)) // Loads-per-thread for A
#define LPTB ((TSK*TSN)/(RTSM*RTSN)) // Loads-per-thread for B

// Use 2D register blocking (further increase in work per thread)
__kernel void matmul(const int M, const int N, const int K,
                      const __global float* A,
                      const __global float* B,
                      __global float* C) {
    
    // Thread identifiers
    const int tidm = get_local_id(0); // Local row ID (max: TSM/WPTM)
    const int tidn = get_local_id(1); // Local col ID (max: TSN/WPTN)
    const int offsetM = TSM*get_group_id(0); // Work-group offset
    const int offsetN = TSN*get_group_id(1); // Work-group offset
 
    // Local memory to fit a tile of A and B
    __local float Asub[TSK][TSM];
    __local float Bsub[TSN][TSK+2];
 
    // Allocate register space
    float Areg;
    float Breg[WPTN];
    float acc[WPTM][WPTN];
 
    // Initialise the accumulation registers
    for (int wm=0; wm<WPTM; wm++) {
        for (int wn=0; wn<WPTN; wn++) {
            acc[wm][wn] = 0.0f;
        }
    }
    
    // Loop over all tiles
    int numTiles = K/TSK;
    for (int t=0; t<numTiles; t++) {
 
        // Load one tile of A and B into local memory
        for (int la=0; la<LPTA; la++) {
            int tid = tidn*RTSM + tidm;
            int id = la*RTSN*RTSM + tid;
            int row = id % TSM;
            int col = id / TSM;
            int tiledIndex = TSK*t + col;
            Asub[col][row] = A[tiledIndex*M + offsetM + row];
            Bsub[row][col] = B[tiledIndex*N + offsetN + row];
        }
        
        // Synchronise to make sure the tile is loaded
        barrier(CLK_LOCAL_MEM_FENCE);
 
        // Loop over the values of a single tile
        for (int k=0; k<TSK; k++) {
 
            // Cache the values of Bsub in registers
            for (int wn=0; wn<WPTN; wn++) {
                int col = tidn + wn*RTSN;
                Breg[wn] = Bsub[col][k];
            }
 
            // Perform the computation
            for (int wm=0; wm<WPTM; wm++) {
                int row = tidm + wm*RTSM;
                Areg = Asub[k][row];
                for (int wn=0; wn<WPTN; wn++) {
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
 
        // Synchronise before loading the next tile
        barrier(CLK_LOCAL_MEM_FENCE);
    }
 
    // Store the final results in C
    for (int wm=0; wm<WPTM; wm++) {
        int globalRow = offsetM + tidm + wm*RTSM;
        for (int wn=0; wn<WPTN; wn++) {
            int globalCol = offsetN + tidn + wn*RTSN;
            C[globalCol*M + globalRow] = acc[wm][wn];
        }
    }
}
"""

In [166]:
gpu_matmul = """
#pragma OPENCL EXTENSION cl_khr_fp64: enable

__kernel void matmul(__global float* C, 
          __global float* A, 
          __global float* B, 
          const int wA, const int wB){
  
   int tile_x = get_global_id(0); 
   int tile_y = get_global_id(1);
 
   // value stores the element that is computed by the thread
   float value = 0;
   for (int k = 0; k < wA; ++k){
      float elementA = A[tile_y * wA + k];
      float elementB = B[k * wB + tile_x];
      value += elementA * elementB;

      barrier(CLK_LOCAL_MEM_FENCE);
   }
 
   // Write the matrix to device memory each thread writes one element
   C[tile_y * wA + tile_x] = value;
}
"""

In [167]:
program = cl.Program(context, gpu_matmul).build()

In [183]:
n = np.int32(2**12)
m = n
k = n

a = np.random.rand(m, n).astype(np.float32)
b = np.random.rand(n, k).astype(np.float32)
c = np.empty_like(np.random.rand(m,k).astype(np.float32))

mf = cl.mem_flags
a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)
c_buf = cl.Buffer(context, mf.WRITE_ONLY, c.nbytes)


TS = 16
local = (TS, TS)
global_size = (m, n)

# local = tuple(map(int, local))
# global_size = tuple(map(int, global_size))


kernel = program.matmul

kernel.set_arg(0, c_buf)
kernel.set_arg(1, a_buf)
kernel.set_arg(2, b_buf)
kernel.set_arg(3, m)
kernel.set_arg(4, k)
# kernel.set_arg(5, c_buf)

event = cl.enqueue_nd_range_kernel(queue, kernel, global_size, local)

# event = program.matmul(queue, global_size, local,
#                         np.int32(a.shape[1]), np.int32(b.shape[1]), np.int32(b.shape[0]),
#                         a_buf, b_buf, c_buf)

event.wait()


In [184]:
cl.enqueue_copy(queue, c, c_buf)
c

array([[1004.858  , 1012.63184, 1006.4273 , ..., 1015.6985 ,  999.89984,
        1030.3223 ],
       [1036.9243 , 1034.8726 , 1042.5879 , ..., 1045.3645 , 1026.7124 ,
        1056.0624 ],
       [1022.2027 , 1016.8986 , 1020.7775 , ..., 1028.5984 , 1021.4653 ,
        1043.6118 ],
       ...,
       [1014.9922 , 1003.9858 , 1014.8939 , ..., 1016.62616, 1008.9029 ,
        1038.7614 ],
       [1028.1812 , 1014.7839 , 1019.60077, ..., 1022.00793, 1022.5033 ,
        1039.1003 ],
       [1022.1451 , 1007.8258 , 1018.5775 , ..., 1018.6975 , 1021.04156,
        1039.0576 ]], dtype=float32)

In [185]:
print(np.dot(a,b))
print(np.matmul(a,b))

[[1004.85785 1012.63104 1006.4269  ... 1015.6977   999.89886 1030.3209 ]
 [1036.9236  1034.873   1042.5878  ... 1045.3646  1026.7124  1056.0621 ]
 [1022.204   1016.8974  1020.77814 ... 1028.5984  1021.4633  1043.615  ]
 ...
 [1014.9924  1003.9858  1014.895   ... 1016.6276  1008.9037  1038.7601 ]
 [1028.1803  1014.782   1019.6004  ... 1022.0074  1022.5029  1039.1    ]
 [1022.14465 1007.82684 1018.57623 ... 1018.697   1021.04034 1039.0581 ]]
[[1004.85785 1012.63104 1006.4269  ... 1015.6977   999.89886 1030.3209 ]
 [1036.9236  1034.873   1042.5878  ... 1045.3646  1026.7124  1056.0621 ]
 [1022.204   1016.8974  1020.77814 ... 1028.5984  1021.4633  1043.615  ]
 ...
 [1014.9924  1003.9858  1014.895   ... 1016.6276  1008.9037  1038.7601 ]
 [1028.1803  1014.782   1019.6004  ... 1022.0074  1022.5029  1039.1    ]
 [1022.14465 1007.82684 1018.57623 ... 1018.697   1021.04034 1039.0581 ]]


In [171]:
np.testing.assert_almost_equal(np.matmul(a,b), c, decimal=2)

In [11]:
gpu_matmul = """
#define BLOCK_SIZE 32
#define global_idx(x_idx, y_idx, m) (x_idx * m + y_idx)

#define WPT 8
#define RBLOCK_SIZE (BLOCK_SIZE/WPT)

// -- Uses NDRange Kernel with Local Memory and 2D Register tiling
// M, N, P can be arbitrary sizes
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE / WPT, BLOCK_SIZE / WPT, 1)))
void matmul(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P,
      __const int M_,
      __const int N_,
      __const int P_)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    
    float Areg;
    float Breg[WPT];
    float acc[WPT][WPT];
    for(int wm=0; wm<WPT; wm++){
        for(int wn=0; wn<WPT; wn++){
            acc[wm][wn] = 0.0f;
        }
    }
    const int numTiles = N_/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        for (int wm=0; wm<WPT; wm++){
            for (int wn=0; wn<WPT; wn++){
                const int r = BLOCK_SIZE*t + row;
                const int c = BLOCK_SIZE*t + col;
                if(((m + wm*RBLOCK_SIZE) < M) && ((c + wn*RBLOCK_SIZE) < N)){
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = A[(m + wm*RBLOCK_SIZE)*N + (c + wn*RBLOCK_SIZE)];
                } else {
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }

                if(((p + wn*RBLOCK_SIZE) < P) && ((r + wm*RBLOCK_SIZE) < N)){
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = B[(r + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)];
                } else {
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){
            for (int wn=0; wn<WPT; wn++){
                Breg[wn] = B_local[k][col + wn*RBLOCK_SIZE];
            }
            for (int wm=0; wm<WPT; wm++){
                Areg = A_local[row + wm*RBLOCK_SIZE][k];
                for (int wn=0; wn<WPT; wn++){
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    for (int wm=0; wm<WPT; wm++){
        for (int wn=0; wn<WPT; wn++){
            if(((m + wm*RBLOCK_SIZE) < M) && ((p + wn*RBLOCK_SIZE) < P)){
                C[(m + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)] = acc[wm][wn];
            }
            
        }
    }
}
"""

In [19]:
# os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0'
# os.environ['PYOPENCL_CTX'] = '0:1'

class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start
        
def roundUp(numToRound, multiple):
    if (multiple == 0):
        return numToRound
    remainder = numToRound % multiple
    if (remainder == 0):
        return numToRound
    return numToRound + multiple - remainder
BLOCK_SIZE = 32
WPT = 8

mult = 2
m, n, k = 3*(10**mult), 4*(10**mult), 5*(10**mult)
m_, n_, k_ = roundUp(m, BLOCK_SIZE), roundUp(n, BLOCK_SIZE), roundUp(k, BLOCK_SIZE)
a = np.random.randn(m, n).astype(np.float32)
b = np.random.randn(n, k).astype(np.float32)

program = cl.Program(context, gpu_matmul).build()

In [20]:
np_times, py_times, cl_times = [], [], []

# ctx = cl.create_some_context()
# queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)

start = time.time()
c = a.dot(b)
end = time.time()
print('np: ', end - start)
np_times.append(end - start)
start = time.time()
c = a @ b
end = time.time()
print('py: ', end - start)
py_times.append(end - start)

c_res = np.zeros_like(c)
c_buf = cl.Buffer(context, mf.WRITE_ONLY, c_res.nbytes)
c.shape

with Timer() as t:
    event = program.matmul(queue, (m_ // WPT , k_ // WPT), (BLOCK_SIZE // WPT, BLOCK_SIZE // WPT), 
                     a_buf, b_buf, c_buf,
                     np.int32(m), np.int32(n), np.int32(k),
                     np.int32(m_), np.int32(n_), np.int32(k_))
    event.wait()
print('cl: ', t.interval)
cl_times.append(t.interval)
cl.enqueue_copy(queue, c_res, c_buf)

c_res.shape, m, n, k

np:  0.0020112991333007812
py:  0.0030014514923095703
cl:  0.007557868957519531


((300, 500), 300, 400, 500)

In [23]:
np.testing.assert_almost_equal(c_res, c, decimal=4)